## Importing Libraries

In [37]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn import tree
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
import seaborn as sns
from sklearn.linear_model import LinearRegression
warnings.filterwarnings("ignore")

# Classification

## Cleaning

In [38]:
difficulty=pd.read_csv("Difficulty2.csv")
difficulty=difficulty.drop(np.arange(0,8))
difficulty=difficulty.drop(np.arange(190,223))
difficulty=difficulty.drop(difficulty.columns[0],axis=1)
difficulty.rename(columns=dict(zip(difficulty.columns, difficulty.loc[8])), inplace=True)
difficulty=difficulty.drop([8,9])
difficulty=difficulty.reset_index().drop("index",axis=1)
difficulty=difficulty.fillna(method="ffill")
difficulty=difficulty.rename(columns={difficulty.columns[4]:"Percentage"})
difficulty=difficulty.drop("Reference period",axis=1)
total_groups=difficulty[difficulty.columns[1]].unique()
age_group=total_groups[1:5]
total_groups
age=difficulty[difficulty["Sociodemographic characteristics"].isin(age_group)].reset_index().drop("index",axis=1)


### Creating a New Dataframe with parameters & predictions

In [39]:
age_classes=age.iloc[age.groupby(["Gender","Sociodemographic characteristics"])["Percentage"].idxmax()]
age_classes_encoded = pd.get_dummies(age_classes, columns=['Gender', 'Sociodemographic characteristics'])
X = age_classes_encoded.drop(columns=['Indicators',"Percentage"])
y = age_classes_encoded['Indicators']
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X, y);
age_classes_encoded.head()

,Indicators,Percentage,Gender_Men,Gender_Women,Sociodemographic characteristics_15 to 24 years,Sociodemographic characteristics_25 to 54 years,Sociodemographic characteristics_55 to 64 years,Sociodemographic characteristics_65 years and over
2,Easy or very easy for household to meet financ...,40.5,True,False,True,False,False,False
3,Very difficult or difficult for household to m...,39.3,True,False,False,True,False,False
7,Neither difficult nor easy for household to me...,36.7,True,False,False,False,True,False
11,Easy or very easy for household to meet financ...,39.3,True,False,False,False,False,True
14,Easy or very easy for household to meet financ...,37.7,False,True,True,False,False,False


Based on this data, there is not a big difference between Genders, but a lot of difference between ages. Ages 15-24 find it easy or very easy for their household to meet financial needs. 25054 seems to find it the hardest, and 55-64 does not find it difficult nor easy. Over 65 seems to find it easy or very easy. It's important to recognise that this data was aggregated before, and does not have individual records due to Statistics Canada. Therefore, this model uses the maximum for each class to predict. Averages do not apply to individuals. Since we do not have individual records, we cannot use cross validation to test accuracy. We will do that elsewhere.

### This is our first classification model predicting based on age & gender.

In [40]:
logistic_regression_model.predict([[True,False,False,False,False,True],
                                   [False,True,False,False,True,False],[True,False,False,True,False,False]])

array(['Easy or very easy for household to meet financial needs 4',
       'Neither difficult nor easy for household to meet financial needs 4',
       'Very difficult or difficult for household to meet financial needs 4'],
      dtype=object)

## Creating another Classification Model based on gender & education level

In [41]:
education_groups = total_groups[21:25]
filtered_df = difficulty[difficulty["Sociodemographic characteristics"].isin(education_groups)]

# Now, group by gender and "Sociodemographic characteristics", and find the row with the maximum percentage within each group
education_classes = filtered_df.loc[filtered_df.groupby(["Gender", "Sociodemographic characteristics"])["Percentage"].idxmax()]
education_classes_encoded = pd.get_dummies(education_classes, columns=['Gender', 'Sociodemographic characteristics'])
X2 = education_classes_encoded.drop(columns=['Indicators',"Percentage",
                                             "Sociodemographic characteristics_Total, highest certificate, diploma or degree"])
y2 = education_classes_encoded['Indicators']
logistic_regression_model2 = LogisticRegression()
logistic_regression_model2.fit(X2, y2);

In [42]:
logistic_regression_model2.predict([[True,False,True,False,False]])

array(['Neither difficult nor easy for household to meet financial needs 4'],
      dtype=object)

In [43]:
education_classes_encoded.drop("Sociodemographic characteristics_Total, highest certificate, diploma or degree",axis=1).head()

,Indicators,Percentage,Gender_Men,Gender_Women,"Sociodemographic characteristics_No certificate, diploma or degree","Sociodemographic characteristics_Postsecondary certificate, diploma or degree",Sociodemographic characteristics_Secondary (high) school diploma or equivalency certificate
67,Neither difficult nor easy for household to me...,37.6,True,False,True,False,False
73,Neither difficult nor easy for household to me...,34.2,True,False,False,True,False
70,Neither difficult nor easy for household to me...,38,True,False,False,False,True
64,Neither difficult nor easy for household to me...,35.7,True,False,False,False,False
156,Very difficult or difficult for household to m...,36.1,False,True,True,False,False
